In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator='\n',
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()

# cache_dir에 있는 embed를 확인하고 업으면 OpenAIEmbeddings 사용
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

# retriever : class의 interface. document를 retrieve
# from vector store, cloud 등
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions.")

'Victory Mansions is a building with glass doors that Winston Smith enters. The hallway has a smell of boiled cabbage and old rag mats. There is a large colored poster on one end of the hallway, depicting the face of a man in his forties with a black mustache and ruggedly handsome features. The building has seven flights of stairs, as the elevator is usually not working and the electricity is cut off during daylight hours. On each landing, there is a poster with the face of Big Brother, with the caption "BIG BROTHER IS WATCHING YOU." Inside the flat, there is a telescreen, an oblong metal plaque on the wall that cannot be completely shut off. The window overlooks a cold and colorless world outside, with little eddies of wind whirling dust and torn paper.'